# Applies Deep Learning to ePodium dataset for prediction of Dyslexia.

#### Import Packages

In [1]:
import os
import matplotlib.pyplot as plt
import ipywidgets

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, BinaryAccuracy, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from functions import epodium_deep_learning
from models.dnn import fully_connected_model
from models.transformer import TransformerModel

import local_paths

#### Choose which processed data to use

In [2]:
processing_method_widget = ipywidgets.RadioButtons(options=['autoreject', 'ransac'], 
                                                   value='autoreject', 
                                                   description='processing:')
display(processing_method_widget)

RadioButtons(description='processing:', options=('autoreject', 'ransac'), value='autoreject')

<br>

## Preparing data as input to the deep learning models.

#### Check number of clean epochs* in each file after processing and split into train and test dataset

*In the context of electroencephalography (EEG), *epochs* are EEG segments in which an event occurs. During processing, the epochs are chosen to be 1 second in which the event occurs at 0.2s. In the context of deep learning, *epochs* are iterations over the entire training dataset.

In [7]:
if(processing_method_widget.value == "autoreject"):
    path_processed = local_paths.ePod_processed_autoreject
if(processing_method_widget.value == "ransac"):
    path_processed = local_paths.ePod_processed_ransac

train, test = epodium_deep_learning.split_train_test_datasets(path_processed)

Analyzed: 220, bad: 40
180 files have enough epochs for analysis.
The dataset is split up into 132 train and 48 test experiments


In [9]:
type(test)

list

#### Create Iterator Sequence as input to feed the model
https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence


In [5]:
train_sequence = epodium_deep_learning.EvokedDataIterator(train)
test_sequence = epodium_deep_learning.EvokedDataIterator(test)

x, y = train_sequence.__getitem__(0)
x.shape

(96, 32, 512)

<br>

## Deep Learning

The data is an *evoked* or *ERP* from a participant in the ePodium experiment. 60 EEG signals were averaged from -0.2 to +0.8 seconds after onset of an event. This is done for each of the 12 event types seperately.

__dimensions__: 
+ x (batches, timesteps, channels)
+ y (batches, labels)

__labels__: 
+ (Sex, At risk of dyslexia, first standard, standard, deviant)


#### Choose Deep Learning model

In [12]:
model_widget = ipywidgets.RadioButtons(options=['fully_connected', 'transformer'],
                                       value='transformer', 
                                       description='processing:')
display(model_widget)

RadioButtons(description='processing:', index=1, options=('fully_connected', 'transformer'), value='transforme…

#### Train model

In [15]:
if(model_widget.value == "fully_connected"): # TODO
    model = fully_connected_model()
    
if(model_widget.value == "transformer"):
    model = TransformerModel()

# Save file with names of test set, so 
path_testset = os.path.join(local_paths.models, model_widget.value + ".txt")

with open(path_testset, 'w') as f:
    for participant in test:
        f.write(participant + '\n')

# with open(the_filename, 'r') as f:
#     my_list = [line.rstrip('\n') for line in f]

    
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=BinaryCrossentropy(),
              metrics=[Precision(), BinaryAccuracy(), Recall()])

model_path = os.path.join(local_paths.models, model_widget.value + ".hdf5")
checkpointer = ModelCheckpoint(filepath = model_path, monitor='val_loss', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, verbose=1)

history = model.fit(x=train_sequence,
                    validation_data=test_sequence,
                    epochs=100,
                    callbacks=[checkpointer, reduce_lr])

Epoch 1/100
 1/17 [>.............................] - ETA: 12:13 - loss: 2.0562 - precision_1: 0.3234 - binary_accuracy: 0.4271 - recall_1: 0.4833

2022-08-31 16:46:50.590489: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: '/volume-ceph/floris_storage/processed/ePod_autoreject/epochs_split/223b_GiepM_FS.npy'
Traceback (most recent call last):

  File "/home/fpauwels/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/fpauwels/.local/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/fpauwels/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1030, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/fpauwels/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 831, in wrapped_generator
    for data in generator_fn():

  File "/home/fpauwels/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", lin

UnknownError: Graph execution error:

FileNotFoundError: [Errno 2] No such file or directory: '/volume-ceph/floris_storage/processed/ePod_autoreject/epochs_split/223b_GiepM_FS.npy'
Traceback (most recent call last):

  File "/home/fpauwels/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/fpauwels/.local/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/fpauwels/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1030, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/fpauwels/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 831, in wrapped_generator
    for data in generator_fn():

  File "/home/fpauwels/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 957, in generator_fn
    yield x[i]

  File 

#### Show Results

In [ ]:
model_extension = '.hdf5'

path_models = glob.glob(os.path.join(local_paths.models, '*' + model_extension))

models = []
paths_model_file = []
paths_model_history = []

for path_model in path_models:
    model_temp = os.path.splitext(path_model)[0]    
    models.append(os.path.basename(model_temp))
    paths_model_file.append(model_temp + model_extension)
    paths_model_history.append(model_temp + '_history.npy')

model = ipywidgets.Select(options=models, description='Models:')
display(model)

In [ ]:
history = np.load(paths_model_history[model.index], allow_pickle=True).item()
display_helper.show_plot(x = range(len(history['loss'])), y = history['loss'] ,xlabel = "epochs", ylabel = "loss")


#### Make predictions on test set